# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Dataset, Experiment
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, sep = '\n')

udacityws
udacitycapstone
japaneast


In [3]:
# choose a name for experiment
experiment_name = 'capstone'
experiment=Experiment(workspace, experiment_name)

dataset = Dataset.get_by_name(workspace, name='Census Income Data Set')
df = dataset.to_pandas_dataframe()
df.head(10)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
5,37,Private,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K
6,49,Private,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.0,0.0,16.0,Jamaica,<=50K
7,52,Self-emp-not-inc,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,45.0,United-States,>50K
8,31,Private,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084.0,0.0,50.0,United-States,>50K
9,42,Private,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.0,0.0,40.0,United-States,>50K


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

### YOUR CODE HERE ###
cpu_cluster_name = "cpu-cluster"

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6, 
                                                           idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(workspace, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "compute_target": cpu_cluster,
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(
    task = 'classification',
    training_data = dataset,
    #valudation_data = validation_data,
    label_column_name = 'income',
    n_cross_validations = 5,
    **automl_settings)

In [6]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster



Experiment,Id,Type,Status,Details Page,Docs Page
capstone,AutoML_9c72d322-9be6-4e20-9c48-f5b7fdfa4420,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms

In [7]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
automl_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
capstone,AutoML_9c72d322-9be6-4e20-9c48-f5b7fdfa4420,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_9c72d322-9be6-4e20-9c48-f5b7fdfa4420',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-19T02:50:22.817563Z',
 'endTimeUtc': '2021-07-19T03:31:16.886912Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone","subscription_id":"653662f1-95de-4498-b876-1fd625bf5d18","resource_group":"udacitycapstone","workspace_name":"udacityws","region":"japaneast","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","da

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
best_automl_run, fitted_automl_model = automl_run.get_output()

print(fitted_automl_model)

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    enable_streaming=None,
    timeseries_param_dict=None,
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, reg_alpha=0, reg_lambda=1.9791666666666667, subsample=0.8, tree_method='auto'))], verbose=False))], flatten_transform=None, weights=[0.5555555555555556, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111]))],
         verbose=False)
Y_transformer(['LabelEncoder', LabelEncoder()])


In [21]:
fitted_automl_model._final_estimator

PreFittedSoftVotingClassifier(
    estimators=[('0', Pipeline(memory=None,
             steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                    ('lightgbmclassifier',
                     LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(
        dataset_samples=45222,
        dataset_features=102,
        dataset_classes=None,
        dataset_num_categorical=0,
        dataset_categoricals=None,
        pipeline_categoricals=None,
        dataset_y_std=None,
        dataset_uid=None,
        subsamp...
        subsampling_schedule='hyperband_clip',
        cost_mode_param=None,
        iteration_timeout_mode=0,
        iteration_timeout_param=None,
        feature_column_names=None,
        label_column_name=None,
        weight_column_name=None,
        cv_split_column_names=None,
        enable_streaming=None,
        timeseries_param_dict=None,
        gpu_training_param_dict={'processing_unit_type': 'cpu'}
    ), random_state=None))],
       

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
print(best_automl_run)

Run(Experiment: capstone,
Id: AutoML_9c72d322-9be6-4e20-9c48-f5b7fdfa4420_65,
Type: azureml.scriptrun,
Status: Completed)


In [11]:
#TODO: Save the best model
import joblib
joblib.dump(fitted_automl_model, './automl_best_model.joblib', compress = 3)

['./automl_best_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [13]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

# Register the model to deploy
model = automl_run.register_model(description = 'Census Income Data Set')
print(f'model ID {automl_run.model_id}')
# Combine scoring script & environment in Inference configuration
env = best_automl_run.get_environment().save_to_directory(path='environments')
script_file= 'score.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)
inference_config = InferenceConfig(entry_script=script_file, environment=env)

model ID AutoML9c72d322965


In [22]:
from azureml.core.webservice import AciWebservice, Webservice

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [23]:
from azureml.core.model import Model
# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = workspace,
    name = "capstone-model",
    models = [model],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite = True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-19 04:25:27+00:00 Creating Container Registry if not exists.
2021-07-19 04:25:29+00:00 Use the existing image.
2021-07-19 04:25:30+00:00 Generating deployment configuration.
2021-07-19 04:25:31+00:00 Submitting deployment to compute..
2021-07-19 04:25:38+00:00 Checking the status of deployment capstone-model..
2021-07-19 04:26:03+00:00 Checking the status of inference endpoint capstone-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [25]:
print(service.state)

Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
import json

input_payload = json.dumps({
    'data': json.loads(df.drop('income', axis = 1)[10:13].to_json(orient='records'))
})
print(f'input : f{input_payload}')
print(f"expected: {df['income'][10:13]}")
output = service.run(input_payload)

print(f'output: {output}')

input : f{"data": [{"age": 37, "workclass": " Private", "education_level": " Some-college", "education-num": 10.0, "marital-status": " Married-civ-spouse", "occupation": " Exec-managerial", "relationship": " Husband", "race": " Black", "sex": " Male", "capital-gain": 0.0, "capital-loss": 0.0, "hours-per-week": 80.0, "native-country": " United-States"}, {"age": 30, "workclass": " State-gov", "education_level": " Bachelors", "education-num": 13.0, "marital-status": " Married-civ-spouse", "occupation": " Prof-specialty", "relationship": " Husband", "race": " Asian-Pac-Islander", "sex": " Male", "capital-gain": 0.0, "capital-loss": 0.0, "hours-per-week": 40.0, "native-country": " India"}, {"age": 23, "workclass": " Private", "education_level": " Bachelors", "education-num": 13.0, "marital-status": " Never-married", "occupation": " Adm-clerical", "relationship": " Own-child", "race": " White", "sex": " Female", "capital-gain": 0.0, "capital-loss": 0.0, "hours-per-week": 30.0, "native-countr

In [18]:
service.get_logs()

'2021-07-19T03:51:13,846754200+00:00 - rsyslog/run \n2021-07-19T03:51:13,847021400+00:00 - gunicorn/run \n2021-07-19T03:51:13,845901000+00:00 - iot-server/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2021-07-19T03:51:13,920598800+00:00 - nginx/run \nrsyslogd: /azureml-envs/azureml_fc92eee9a5613508afa12283dd0b27d8/lib/libuuid.so.1: no version information available (required by rsyslogd)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-07-19T03:51:14,295570800+00:00 - iot-server/finish 1 0\n2021-07-19T03:51:14,303021400+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (64)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 92\nSPARK_HOME not set. Skipping PySpark Initialization.\nGenerating new fontManager, this may take some time...\nInitializing logger\n2021-07-19 03:51:17,171 | root | INFO | Starting up app insights client\

TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
service.delete()